

### 1) 연신Lot ~ 출하 Lot/ 원재료Lot ~ 연신Lot 합치기 ###

>- 대상 : 20160329BE01004/20160329BE01005/20160329BE01006 3개 Lot
 
>- 결과물 : 엑셀 파일

### 2) 연신 ~ 출하 Lot 진행 사항 추적 ###

>- 대상 : 20160917CE02005 1개 Lot

>- 결과물 : 엑셀 파일

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 12 19:52:00 2016

@author: whikwon
"""

import pandas as pd
import re
import os
import numpy as np

Folder = 'D:\\#.Secure Work Folder\\3.이슈\\X417\\170109_X417상 얼룩 이슈'

os.chdir(Folder)

In [6]:
# Lot, Code, Qty 쪼개는 Logic
foo = lambda x : pd.Series([i for i in re.split('[_]|[(]|[)]', str(x), 3)[0:3]])

# 엑셀 불러오기 
down = pd.read_excel('Lot정보.xlsx', sheetname = 'Sheet1', parse_cols = ('A,B,E:T'))

# 전부다 공백인 줄 삭제 
down.dropna(axis = 0, how = 'all', inplace = True)

# 연신 Lot 기준, 전체 채우기
down['기준LOT'] = down['기준LOT'].fillna(method = 'ffill', inplace = True)

# 빈 Dataframe 만들기
DOWN = pd.DataFrame()

# 총 연신Lot List 뽑기
Lot = down['기준LOT'].drop_duplicates()

# 1줄 씩 각 연신 별로 Loop 돌려서 맨 뒤 위치를 보고 그에 해당되는 바로 윗 줄 fillna 한다.  
for i in Lot:
    a = down[down['기준LOT'] == i]
    a.reset_index(inplace = True, drop = True)
    for k in range(len(a)-1):
        for j, l in zip(reversed(a.ix[k+1,:]), reversed(range(len(a.ix[k+1,:])))):
            if type(j) == str:
                a.ix[k+1,:l] = a.ix[k+1,:l].fillna(a.ix[k,:l])
                break
        
    DOWN = pd.concat([DOWN, a], axis = 0)
    
# 빈 Dataframe 만들기 2
data = pd.DataFrame()

# Lot, Code, Qty 쪼개서 새로 입력 하고 합침. 
for i in DOWN.columns:
    A = DOWN[i].apply(foo)
    data = pd.concat([data, A], axis = 1)

# 수량에 해당되는 값 int로 변환 함. 여기까지만 하면 전체 Process 상세 정보 나옴. 
data.ix[:,2] = data.ix[:,2].fillna(0).astype('int')
data.iloc[:,2] = data.iloc[:,2].astype('float')
data.to_excel('공정진행실적.xlsx', index = False)

공통된 부분 및 검사Lot ~ 출하Lot까지 조회

In [7]:
# 아래에서 index에 매칭하기 위해서 reset해준다.

data_1 = data

data_1.reset_index(drop = True, inplace = True)

# 검사Lot 찾는 Logic
insp_find = re.compile('.{8}[B,C,D][S].{5}') 


'''
아래 코드가 의미하는 거 :

2번째 줄의 iloc[:, :24] 는 어디까지 재단인지(process가 똑같은 지점)를 나타내준다. 
3번째 줄의 range는 검사 Lot일 수 있는 Column의 range를 나타낸다. 3개 column 주기로 확인하면 됨. 

'''
# X417은 타 모델이랑 달라짐. ※ 참조.. X417 : (:24), (9,25,3) ※ RC40 : [:, :15], (1, 35, 3)
to_cutting = data_1.iloc[:,:15]

# 재단까지 정보 , C는 재단 이후 정보 합치려고 만든 Dataframe 
after_cutting = pd.DataFrame()

for i in [-i for i in range(1, 35, 3)]:
    data_1 = data_1.astype('str')
    B = data_1.iloc[:,i:][data_1.iloc[:,i].apply(lambda x: bool(re.match(insp_find, x)))]
    B.columns = [i for i in range(50, 50+(len(B.columns)))]
    data_1 = data_1.drop(B.index)    
    after_cutting = pd.concat([after_cutting,B], axis = 0)

to_cutting = pd.merge(to_cutting, after_cutting, left_index = True, right_index = True)

to_cutting.iloc[:,range(2,len(to_cutting.columns),3)] = to_cutting.iloc[:,range(2,len(to_cutting.columns),3)].fillna(0).astype(float)
    

IndexError: single positional indexer is out-of-bounds

엑셀로 검사 진행 실적/미 진행 실적 불러오기

In [10]:
to_cutting.to_excel('검사진행실적.xlsx', index = True)

data_1.to_excel('검사미진행실적.xlsx', index = False)    

연신 → 원재료/ 연신 → 검사 실적 합치기

In [54]:
raw_1 = pd.read_excel('D:\\#.Secure Work Folder\\구미CS\\Python 교육 자료\\raw-data.xlsx', sheetname = '연신-검사')
raw_1.rename(columns = {'추적LOT':'연신','투입LOT':'재단','투입제품코드':'재단code','생산LOT':'검사','생산제품코드':'검사code'}, inplace = True)
raw_2 = pd.read_excel('D:\\#.Secure Work Folder\\구미CS\\Python 교육 자료\\raw-data.xlsx', sheetname = '원재료-연신', names = ['연신','연신code','원재료1','원재료code','원재료명','원재료2'])
raw_2.rename(columns = {'추적LOT':'연신','생산제품코드':'연신code','투입LOT':'원재료1','투입제품코드':'원재료code','투입제품명':'원재료명','투입비고':'원재료2'}, inplace = True)

raw_data = pd.merge(raw_2, raw_1, how = 'inner', on = '연신')
raw_data.to_excel('D:\\#.Secure Work Folder\\구미CS\\Python 교육 자료\\원재료-연신-검사.xlsx', index = False)
